# The price of state - NOW RAG

| Phase       | What we build                   |
| ----------- | -------------------------------- |
| Phase 1     | RAG for comparables              |
| Phase 2     | ML + RAG blended estimator       |
| Phase 3     | Agent + strategy                 |
| Phase 4     | UI                               |


Dự án này minh hoạ một trợ lý định giá theo mô hình agentic dành cho chủ nhà Airbnb, kết hợp:

- Ước tính giá dựa trên Machine Learning (ML)
- Retrieval-Augmented Generation (RAG) sử dụng các listing tương tự
- Suy luận bằng LLM để xây dựng chiến lược định giá và giải thích đề xuất

Hệ thống được thiết kế như một công cụ hỗ trợ ra quyết định, thay vì chỉ là một mô hình dự đoán thuần túy, với trọng tâm là tính minh bạch, sự phù hợp với thị trường và khả năng ứng dụng thực tế.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load dataset from phase 2
df_phase3 = pd.read_pickle("/Users/macos/projects/aicourse/week1/df_phase2.pkl")

In [3]:
# 1. Choose what goes into embeddings
EMBED_TEXT_COLS = [
    "final_text_clean"
]

df_rag = df_phase3[
    ["id", "price", "latitude", "longitude", "room_type", "property_type", "final_text_clean"]
].dropna(subset=["final_text_clean", "price"])

## Vector Retrieval (Chroma + SentenceTransformer)

all-MiniLM là một mô hình rất hữu ích từ HuggingFace, có khả năng chuyển đổi câu và đoạn văn thành vector 384 chiều, rất phù hợp cho các tác vụ như semantic search (tìm kiếm ngữ nghĩa).

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Mô hình này có thể chạy khá nhanh trên máy local.

Ngoài ra, OpenAI cũng cung cấp một mô hình Embeddings dạng closed-source. Tuy nhiên, so với embeddings của OpenAI thì all-MiniLM có một số lợi thế:

Miễn phí!
Có thể chạy hoàn toàn trên máy local, dữ liệu không rời khỏi hệ thống của bạn — điều này đặc biệt hữu ích nếu bạn đang xây dựng một hệ thống RAG cá nhân

In [4]:
# 2. SentenceTransformer encoder

encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [5]:
import chromadb
from tqdm import tqdm

# Khởi tạo ChromaDB client - lưu trữ vĩnh viễn
client = chromadb.PersistentClient(path="vectorstore")
collection = client.get_or_create_collection("bangkok_airbnb")

In [ ]:
# Embed và Lưu Data vào ChromaDB -- (run 1 time)

BATCH = 512

for i in tqdm(range(0, len(df_rag), BATCH)):
    batch = df_rag.iloc[i:i+BATCH]

    documents = batch["final_text_clean"].tolist()
    embeddings = encoder.encode(documents).astype(float).tolist()

    metadatas = [
        {
            "price": float(p),
            "room_type": rt,
            "property_type": pt,
            "lat": lat,
            "lon": lon
        }
        for p, rt, pt, lat, lon in zip(
            batch.price,
            batch.room_type,
            batch.property_type,
            batch.latitude,
            batch.longitude
        )
    ]

    ids = [f"listing_{idx}" for idx in batch.index]

    collection.add(
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )
  # --> Now we have a semantic Airbnb market memory

100%|██████████| 46/46 [20:02<00:00, 26.13s/it]


In [7]:
# Retrieval: “Find comparable listings”
from functools import lru_cache

@lru_cache(maxsize=1024)
def encode_text(text: str):
    return encoder.encode([text])[0]

def retrieve_similar_listings(text, k=7):
    vec = encode_text(text)
    results = collection.query(
        query_embeddings=[vec.astype(float).tolist()],
        n_results=k
    )
    docs = results["documents"][0]
    prices = [m["price"] for m in results["metadatas"][0]]
    return docs, np.array(prices)

In [8]:
# test it -- aha moment
docs, prices = retrieve_similar_listings(df_rag.iloc[0].final_text_clean)
prices

array([1595.,  691., 1415., 2377., 1062., 1500.,  727.])

In [9]:
# Turning retrieval into a RAG prompt - Tạo Context cho LLM

def make_airbnb_context(docs, prices):
    msg = "Here are similar Airbnb listings in Bangkok:\n\n"
    for d, p in zip(docs, prices):
        msg += f"Listing description:\n{d[:300]}...\nPrice: {p:.0f} THB/night\n\n"
    return msg

In [10]:
# The Airbnb Price RAG Agent (LLM) - Prompt for LLM

def messages_for_listing(listing_text, docs, prices):
    msg = (
        "Estimate a reasonable nightly Airbnb price range in THB.\n"
        "Respond in JSON with EXACTLY this format:\n"
        "{\n"
        '  "low": number,\n'
        '  "mid": number,\n'
        '  "high": number\n'
        "}\n"
        "Do NOT include any text, explanation, or currency symbols.\n\n"
        "Target listing:\n"
        f"{listing_text}\n\n"
    )

    msg += make_airbnb_context(docs, prices)

    return [{"role": "user", "content": msg}]

In [11]:
def price_stats(prices):
    return {
        "p25": np.percentile(prices, 25),
        "median": np.median(prices),
        "p75": np.percentile(prices, 75)
    }

In [12]:
# Call LLM (OpenAI / Ollama / GPT-4o-mini / llama): RAG Agent - Kết hợp Retrieval + LLM
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv() 

client = OpenAI()

def airbnb_price_rag(listing_text):
    docs, prices = retrieve_similar_listings(listing_text)
    messages = messages_for_listing(listing_text, docs, prices)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        # reasoning_effort="none"
    )

    return response.choices[0].message.content

In [26]:
# Lấy một listing để test
test_listing = df_rag.iloc[0]
test_text = test_listing["final_text_clean"]
actual_price = test_listing["price"]

print("=" * 60)
print("🏠 LISTING MÔ TẢ:")
print(test_text[:300] + "...")
print(f"\n💰 GIÁ THỰC TẾ: {actual_price:.0f} THB/night")
print("=" * 60)

# Tìm listings tương tự
print("\n🔍 TÌM 5 LISTINGS TƯƠNG TỰ:")
docs, prices = retrieve_similar_listings(test_text, k=5)
for i, (doc, price) in enumerate(zip(docs, prices), 1):
    print(f"\n{i}. Giá: {price:.0f} THB")
    print(f"   Mô tả: {doc[:150]}...")

# Dự đoán giá bằng RAG
print("\n" + "=" * 60)
print("🤖 AI ĐANG DỰ ĐOÁN GIÁ...")
predicted_price = airbnb_price_rag(test_text)
print(f"💡 GIÁ DỰ ĐOÁN: {predicted_price} THB/night")
print(f"💰 GIÁ THỰC TẾ: {actual_price:.0f} THB/night")

import json

try:
    pred_json = json.loads(predicted_price)

    low = float(pred_json["low"])
    mid = float(pred_json["mid"])
    high = float(pred_json["high"])

    # error theo mid price
    error = abs(mid - actual_price) / actual_price * 100

    print(f"📊 LOW: {low:.0f} | MID: {mid:.0f} | HIGH: {high:.0f}")
    print(f"📊 SAI SỐ (MID): {error:.1f}%")

except Exception as e:
    print("⚠️  Không parse được JSON:", e)

🏠 LISTING MÔ TẢ:
We stayed in the apartment for a week and we enjoyed it very much. Nuttee is a very nice host, and she did her best to accommodate us. Everything is perfect in the apartment, and we love the view from the balcony. The apartment is very modern and spacious, and the location is very central, 10 mins w...

💰 GIÁ THỰC TẾ: 1595 THB/night

🔍 TÌM 5 LISTINGS TƯƠNG TỰ:

1. Giá: 1595 THB
   Mô tả: We stayed in the apartment for a week and we enjoyed it very much. Nuttee is a very nice host, and she did her best to accommodate us. Everything is p...

2. Giá: 691 THB
   Mô tả: The host was extremely welcoming and generously helpful! We felt at home in the condo and the area straight away. We were met at the train station nea...

3. Giá: 1415 THB
   Mô tả: This a wonderful little apartment with a great view of Bangkok skyline. It has all the necessary amenities. The building itself is located in an area ...

4. Giá: 2377 THB
   Mô tả: We loved our stay with Nut in Chinatown! The lo

In [ ]:
# Test trên nhiều listings ---> REMOVE
import re

def extract_price(pred): # in case output is NOT number only
    match = re.search(r"[\d,.]+", pred)
    if match:
        return float(match.group(0).replace(",", ""))
    else:
        raise ValueError(f"Could not extract price from: {pred}")

errors = []

for i in range(100):
    listing = df_rag.iloc[i]
    pred = airbnb_price_rag(listing.final_text_clean)

    pred_price = extract_price(pred)
    error = abs(pred_price - listing.price)

    errors.append(error)

mae = np.mean(errors)
print(f"Average error: ฿{mae:.0f}")


Average error: ฿987


# Phase 2: Blending ML + RAG

ML side

model_tfidf → predicts log(price)

MAE ≈ 1,267 THB

RAG side

retrieve_similar_listings(text)

Comparable prices list: [p1, p2, ..., pk]

LLM price reasoning (optional)

In [14]:
# --- Bring back tf-idf + Ridge 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error

# Core sklearn pipeline
from sklearn.pipeline import Pipeline

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Text vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

y = np.log1p(df_phase3["price"])

## 1. Numeric feature preparation
# chỉ chọn features có ý nghĩa kinh tế
NUMERIC_COLS = [
    "accommodates",
    "bedrooms",
    "beds",
    "minimum_nights",
    "number_of_reviews",
    "reviews_per_month",
    "latitude",
    "longitude"
]

df_num = df_phase3[NUMERIC_COLS].copy()

# numeric process pipeline
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

## 2. text data
texts = df_phase3["final_text_clean"].values

X_num_train, X_num_test, X_txt_train, X_txt_test, y_train, y_test = train_test_split(
    df_num,
    texts,
    y,
    test_size=0.2,
    random_state=42
)

X_num_train_proc = numeric_pipeline.fit_transform(X_num_train)
X_num_test_proc  = numeric_pipeline.transform(X_num_test)

baseline_model = Ridge(alpha=1.0)
baseline_model.fit(X_num_train_proc, y_train)

y_pred = baseline_model.predict(X_num_test_proc)
mae_numeric = mean_absolute_error(
    np.expm1(y_test),
    np.expm1(y_pred)
)

print(f"Numeric only MAE: {mae_numeric:,.0f} THB")

tfidf = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    stop_words="english",
    min_df=5
)

X_txt_train_tfidf = tfidf.fit_transform(X_txt_train)
X_txt_test_tfidf  = tfidf.transform(X_txt_test)

# combine numeric + text
from scipy.sparse import hstack

X_train_all = hstack([X_num_train_proc, X_txt_train_tfidf])
X_test_all  = hstack([X_num_test_proc,  X_txt_test_tfidf])

model_tfidf = Ridge(alpha=1.0)
model_tfidf.fit(X_train_all, y_train)

y_pred = model_tfidf.predict(X_test_all)

mae_tfidf = mean_absolute_error(
    np.expm1(y_test),
    np.expm1(y_pred)
)

print(f"Numeric + TF-IDF MAE: {mae_tfidf:,.0f} THB")

Numeric only MAE: 884 THB
Numeric + TF-IDF MAE: 592 THB


In [27]:
def ml_price_predict(listing_text, numeric_df):
    if isinstance(numeric_df, pd.Series):
        numeric_df = numeric_df.to_frame().T
        
    num_proc = numeric_pipeline.transform(numeric_df)
    txt_vec = tfidf.transform([listing_text])
    X = hstack([num_proc, txt_vec])
    log_pred = model_tfidf.predict(X)[0]
    return np.expm1(log_pred)

In [16]:
# Comparable-based distribution

def rag_price_stats(listing_text):
    _, prices = retrieve_similar_listings(listing_text)
    
    return {
        "low": np.percentile(prices, 25),
        "mid": np.median(prices),
        "high": np.percentile(prices, 75),
        "mean": prices.mean(),
        "std": prices.std()
    }

In [ ]:
#---------------

Core idea: ML = anchor, RAG = correction

This is the professional mental model:

ML says where the market usually is
RAG says where this listing fits locally

In [23]:
# Weighted average

def blend_prices(ml_price, rag_stats, w_ml=0.6):
    w_rag = 1 - w_ml
    
    blended_mid = (
        w_ml * ml_price +
        w_rag * rag_stats["mid"]
    )
    
    return {
        "low": max(0, blended_mid - rag_stats["std"]),
        "mid": blended_mid,
        "high": blended_mid + rag_stats["std"]
    }

In [24]:
# pricing function

def price_phase2(listing_text, numeric_row):
    ml_price = ml_price_predict(listing_text, numeric_row)
    rag_stats = rag_price_stats(listing_text)
    
    blended = blend_prices(ml_price, rag_stats)
    
    return {
        "ml_price": ml_price,
        "rag_median": rag_stats["mid"],
        "final_range": blended
    }

In [28]:
# Test blending

print("\n" + "=" * 80)
print("Testing ML + RAG Blending")
print("=" * 80)

test_row = df_phase3.iloc[10]
test_result = price_phase2(
    listing_text=test_row.final_text_clean,
    numeric_row=test_row[NUMERIC_COLS]
)

print(f"\n🤖 ML prediction: {test_result['ml_price']:,.0f} THB")
print(f"🔍 RAG median: {test_result['rag_median']:,.0f} THB")
print(f"📊 Final range: {test_result['final_range']['low']:,.0f} - {test_result['final_range']['high']:,.0f} THB")
print(f"   (recommended: {test_result['final_range']['mid']:,.0f} THB)")
print(f"\n✅ Actual price: {test_row.price:,.0f} THB")



Testing ML + RAG Blending

🤖 ML prediction: 1,248 THB
🔍 RAG median: 1,360 THB
📊 Final range: 865 - 1,721 THB
   (recommended: 1,293 THB)

✅ Actual price: 1,700 THB


# Phase 3: Agent
“How should we price given goals and market context?”

We model pricing as a decision problem, not a regression task.
ML + RAG generate market intelligence, and an agent applies pricing strategy based on host goals and market volatility.

In [ ]:
# from pydantic import BaseModel
# from typing import Literal

# class PricingDecision(BaseModel):
#     strategy: Literal["aggressive", "balanced", "premium"]
#     reasoning: str

In [ ]:
# def strategy_prompt(market_info):
#     r = market_info["final_range"]

#     return f"""
# You are a pricing strategist for Airbnb hosts.

# Market analysis:
# - ML estimated price: {market_info['ml_price']:.0f} THB
# - Comparable median price: {market_info['rag_median']:.0f} THB
# - Market price range: {r['low']:.0f} – {r['high']:.0f} THB

# Choose ONE pricing strategy:
# - aggressive: price near the low end to maximize occupancy
# - balanced: price near the median
# - premium: price near the high end to maximize revenue

# Respond in JSON with fields:
# - strategy
# - reasoning
# """

In [ ]:
# from openai import OpenAI

# client = OpenAI()

# def llm_choose_strategy(market_info):
#     response = client.chat.completions.parse(
#         model="gpt-4o-mini",
#         messages=[{"role": "user", "content": strategy_prompt(market_info)}],
#         response_format=PricingDecision
#     )
#     return response.choices[0].message.parsed

In [ ]:
# def apply_strategy(market_info, strategy):
#     r = market_info["final_range"]

#     if strategy == "aggressive":
#         return r["low"]
#     elif strategy == "premium":
#         return r["high"]
#     else:
#         return r["mid"]

In [ ]:
# class LLMPricingAgent:
#     def __init__(self):
#         pass

#     def price(self, listing_text, numeric_row):
#         # Phase 2
#         market = price_phase2(listing_text, numeric_row)

#         # LLM decision
#         decision = llm_choose_strategy(market)

#         # Apply strategy
#         final_price = apply_strategy(market, decision.strategy)

#         return {
#             "recommended_price": round(final_price),
#             "strategy": decision.strategy,
#             "reasoning": decision.reasoning,
#             "market_context": market
#         }

In [ ]:
# agent = LLMPricingAgent()

# agent.price(
#     test_row.final_text_clean,
#     test_row[NUMERIC_COLS]
# )

{'recommended_price': 1293,
 'strategy': 'balanced',
 'reasoning': 'A balanced pricing strategy, positioned near the median price of 1360 THB, allows us to remain competitive in the market while avoiding the risks associated with aggressive pricing that could undervalue the listing. This approach aims to attract a reasonable number of bookings, ensuring consistent occupancy levels, while also reflecting the quality and value of the property versus its comparables.',
 'market_context': {'ml_price': np.float64(1248.3239509700784),
  'rag_median': np.float64(1360.0),
  'final_range': {'low': np.float64(864.6709185135935),
   'mid': np.float64(1292.994370582047),
   'high': np.float64(1721.3178226505004)}}}

In [ ]:
# print(test_row.final_text_clean)

The place is very nice. Theres a 7-eleven nearby, so its quite convenient. I felt relaxed just looking out the window. Overall, it was a wonderful experience. I feel very homey. The house is close to the Main Street, and easy to get public transportation-the delicious Thai breakfast from the lovely host. I recommend staying in Primroses place. I stayed at Primrose Place for one month at the beginning of my exchange semester at Mahidol University, which is near the AirBnB. My stay at Primrose Place was very nice. Primrose' Mother lives next door to the AirBnB, she was really helpful during my stay. The street next to the Airbnb is pretty calm and there is a 7/11 and a fruit stand right down the street, as well as some restaurants and a place to do laundry. The room is just like it looks in the pictures. I recommend coming here, especially if you are planning to stay for a month or more, for which the price is super reasonable, especially as the electricity and water bills are included. 

# Agent

In [35]:
#---------------

In [ ]:
# decision engine 

def choose_strategy(market):
    ml = market["ml_price"]
    mid = market["rag_median"]

    if ml < mid * 0.9:
        return "aggressive", "ML price is significantly below market median."
    elif ml > mid * 1.1:
        return "premium", "ML price is well above comparable listings."
    else:
        return "balanced", "ML price aligns closely with market median."

In [18]:
# apply strategy

def apply_strategy(market, strategy):
    r = market["final_range"]
    return {
        "aggressive": r["low"],
        "balanced": r["mid"],
        "premium": r["high"]
    }[strategy]

In [20]:
# decision engine class

class DecisionEngine:
    def price(self, listing_text, numeric_row):
        ml_price = ml_price_predict(listing_text, numeric_row)
        rag_stats = rag_price_stats(listing_text)


        blended_mid = 0.6 * ml_price + 0.4 * rag_stats["mid"]

        market = {
            "ml_price": ml_price,
            "rag_median": rag_stats["mid"],
            "final_range": {
                "low": max(0, blended_mid - rag_stats["std"]),
                "mid": blended_mid,
                "high": blended_mid + rag_stats["std"]
            }
         }


        strategy, rule_reason = choose_strategy(market)
        final_price = apply_strategy(market, strategy)


        return {
            "recommended_price": round(final_price),
            "strategy": strategy,
            "rule_reasoning": rule_reason,
            "market": market
        }

In [19]:
from openai import OpenAI
client = OpenAI()


class ExplanationEngine:
    def explain(self, decision):
        prompt = f"""
Explain this Airbnb pricing decision clearly to a host.


Strategy: {decision['strategy']}
ML estimate: {decision['market']['ml_price']:.0f} THB
Market median: {decision['market']['rag_median']:.0f} THB
Range: {decision['market']['final_range']['low']:.0f}–{decision['market']['final_range']['high']:.0f} THB


Do NOT change the price.
Keep it short and practical.
"""

        try:
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                timeout=5
            )
            return resp.choices[0].message.content
        except:
            return decision["rule_reasoning"]

In [ ]:
# complete agent

class AirbnbPricingAgent:
    def __init__(self, explain=True):
        self.decision_engine = DecisionEngine()
        self.explainer = ExplanationEngine() if explain else None

    def price(self, listing_text, numeric_row):
        decision = self.decision_engine.price(listing_text, numeric_row)

        reasoning = (
            self.explainer.explain(decision)
            if self.explainer else decision["rule_reasoning"]
        )

        return {
            "price": decision["recommended_price"],
            "strategy": decision["strategy"],
            "reasoning": reasoning,
            "market": decision["market"]
        }

In [29]:
agent = AirbnbPricingAgent()

agent.price(
    test_row.final_text_clean,
    test_row[NUMERIC_COLS]
)

{'price': 1293,
 'strategy': 'balanced',
 'reasoning': "Hello [Host's Name],\n\nWe're adopting a balanced pricing strategy for your listing. Here's a quick breakdown:\n\n- **ML Estimate**: Our model suggests a competitive price of 1248 THB, which takes into account various factors like your property's features and booking trends.\n- **Market Median**: The median price for similar listings in your area is 1360 THB, indicating what most hosts are charging.\n- **Price Range**: The market range is between 865 THB and 1721 THB, providing flexibility for adjustments in the future.\n\nBy maintaining your current price, you can position your listing attractively while ensuring you're in line with the market. \n\nThank you for your understanding!\n\nBest,  \n[Your Name]  \nAirbnb Team",
 'market': {'ml_price': np.float64(1248.3239509700784),
  'rag_median': np.float64(1360.0),
  'final_range': {'low': np.float64(864.6709185135935),
   'mid': np.float64(1292.994370582047),
   'high': np.float64(

# UI

In [46]:
# ui adapter function

def build_numeric_df(values: dict) -> pd.DataFrame:
    """
    Ensure numeric features match training columns & order
    """
    return pd.DataFrame(
        [[values[col] for col in NUMERIC_COLS]],
        columns=NUMERIC_COLS
    )

agent = AirbnbPricingAgent(explain=True)

def price_room_ui(
    description,
    accommodates,
    bedrooms,
    beds,
    latitude,
    longitude,
    minimum_nights,
    number_of_reviews,
    reviews_per_month,
):
    try:
        values = {
            "accommodates": accommodates,
            "bedrooms": bedrooms,
            "beds": beds,
            "latitude": latitude,
            "longitude": longitude,
            "minimum_nights": minimum_nights,
            "number_of_reviews": number_of_reviews,
            "reviews_per_month": reviews_per_month,
        }

        numeric_df = build_numeric_df(values)

        result = agent.price(description, numeric_df)

        return (
            f"💰 {result['price']:,.0f} THB / night",
            result["strategy"].capitalize(),
            result["reasoning"],
            f"""
**ML estimate:** {result['market']['ml_price']:,.0f} THB  
**Market median:** {result['market']['rag_median']:,.0f} THB  
**Range:** {result['market']['final_range']['low']:,.0f}
– {result['market']['final_range']['high']:,.0f} THB
"""
        )

    except Exception as e:
        return (
            "❌ Error",
            "-",
            str(e),
            ""
        )

In [47]:
with gr.Blocks(title="🏠 Airbnb Host Pricing Assistant") as demo:
    gr.Markdown("## 🏠 Airbnb Host Pricing Assistant")
    gr.Markdown("ML + Market pricing with AI explanations")

    description = gr.Textbox(
        label="Room Description",
        lines=6,
        placeholder="Describe your Airbnb listing..."
    )

    with gr.Row():
        accommodates = gr.Number(label="Accommodates", value=2)
        bedrooms = gr.Number(label="Bedrooms", value=1)
        beds = gr.Number(label="Beds", value=1)

    with gr.Row():
        latitude = gr.Number(label="Latitude", value=13.75)
        longitude = gr.Number(label="Longitude", value=100.50)

    with gr.Row():
        minimum_nights = gr.Number(label="Minimum nights", value=1)
        number_of_reviews = gr.Number(label="Number of reviews", value=10)
        reviews_per_month = gr.Number(label="Reviews per month", value=1.0)

    btn = gr.Button("Recommend Price 🚀")

    price_out = gr.Textbox(label="Recommended Price")
    strategy_out = gr.Textbox(label="Strategy")
    reasoning_out = gr.Textbox(label="Reasoning", lines=4)
    context_out = gr.Markdown()

    btn.click(
        price_room_ui,
        inputs=[
            description,
            accommodates, bedrooms, beds,
            latitude, longitude,
            minimum_nights,
            number_of_reviews,
            reviews_per_month,
        ],
        outputs=[
            price_out,
            strategy_out,
            reasoning_out,
            context_out
        ],
    )

demo.queue()
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
